<a href="https://colab.research.google.com/github/Newball12/tuplas2/blob/main/AV6P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:

# Importaciones necesarias
import pandas as pd
import dask.dataframe as dd
import plotly.express as px
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Descargar recursos NLTK necesarios
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Cargar los datos con Dask
df = dd.read_csv('tweets_dataset.csv')

# Función para analizar sentimientos
def analyze_sentiment(text_series):
    """
    Analiza los sentimientos de una serie de texto utilizando VADER.

    Args:
        text_series (pd.Series): Una serie de Pandas con texto.

    Returns:
        pd.Series: Serie con los sentimientos clasificados ('positivo', 'negativo', 'neutral').
    """
    analyzer = SentimentIntensityAnalyzer()
    results = text_series.apply(lambda text: analyzer.polarity_scores(text)['compound'])
    sentiment_categories = results.apply(lambda score: 'positivo' if score >= 0.05 else
                                                         ('negativo' if score <= -0.05 else 'neutral'))
    return sentiment_categories

# Aplicar análisis de sentimientos con Dask
df['polaridad'] = df['texto'].map_partitions(analyze_sentiment, meta=('polaridad', 'object'))

# Convertir a Pandas para visualización y análisis
df_computed = df.compute()

# 1. Visualización de la distribución de sentimientos
sentiment_counts = df_computed['polaridad'].value_counts().reset_index()
sentiment_counts.columns = ['Polaridad_sentimientos', 'Conteo']
fig_bar = px.bar(sentiment_counts, x='Polaridad_sentimientos', y='Conteo', title='Distribución de Sentimientos en Tweets')
fig_bar.show()

# 2. Análisis temporal: Sentimientos por fecha
df_computed['fecha'] = pd.to_datetime(df_computed['fecha'])

# Agrupar por fecha y contar la frecuencia de cada sentimiento
# Changed 'sentimiento' to 'polaridad' in the line below
df_grouped = df_computed.groupby('fecha')['polaridad'].value_counts().unstack()
print(df_grouped.head())
# Crear un gráfico de barras
fig = px.bar(df_grouped, x=df_grouped.index, y=['negativo', 'neutral',], barmode='stack')
fig.show()

# 3. Filtros interactivos con Plotly (Ejemplo básico de dispersión)
fig_scatter = px.scatter(df_computed, x='fecha', y='polaridad', color='polaridad',
                         hover_data=['texto'], title='Sentimientos de Tweets')

# Control deslizante para filtrar por fecha
fig_scatter.update_layout(
    xaxis_range=[df_computed['fecha'].min(), df_computed['fecha'].max()],
    xaxis_rangeslider_visible=True
)
fig_scatter.show()
print(df_grouped.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


polaridad            negativo  neutral
fecha                                 
2023-11-01 03:40:00      <NA>        1
2023-11-01 05:58:00      <NA>        1
2023-11-01 07:37:00      <NA>        1
2023-11-01 18:47:00      <NA>        1
2023-11-01 19:07:00         1     <NA>


polaridad            negativo  neutral
fecha                                 
2023-11-01 03:40:00      <NA>        1
2023-11-01 05:58:00      <NA>        1
2023-11-01 07:37:00      <NA>        1
2023-11-01 18:47:00      <NA>        1
2023-11-01 19:07:00         1     <NA>
